In [2]:
#import all packages
import pandas as pd
import numpy as np
import re as re
import nltk
from nltk.stem.porter import *

from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
# datafile
data_file = "ShopeeData/SHOPEE_MAYBELLINE.csv"
data = pd.read_csv(data_file)
data.columns = data.columns.str.strip().str.replace(" ","_") # reason for doing this is cause cannot get columns with space into a list

#reformat the respective columns

#lower case
data['Review'] = [word.lower() for word in data['Review']]

#Strip off the Variation Word
data['Product_Purchase'] = data['Product_Purchase'].str.strip("Variation:")

#Change Date of Review to DateTime
data['Date_Of_Review'] = pd.to_datetime(data['Date_Of_Review'])

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2422 entries, 0 to 2421
Data columns (total 11 columns):
Unnamed:_0          2422 non-null int64
Brand               2422 non-null object
Category            2422 non-null object
Product_Name        2422 non-null object
Price               2422 non-null float64
Reviewer            2419 non-null object
Review              2422 non-null object
Product_Purchase    2324 non-null object
Ratings             2419 non-null float64
Date_Of_Review      2419 non-null datetime64[ns]
Response            2419 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 208.2+ KB


,Unnamed:_0,Brand,Category,Product_Name,Price,Reviewer,Review,Product_Purchase,Ratings,Date_Of_Review,Response
0,0,Maybelline,Lips,Maybelline Lip Studio Color Jolt Shiny + Matte...,9.9,peachiii,fast delivery and bubble wrapped. i have yet t...,Bittersweet Mand,5.0,2019-11-24 14:32:00,no
1,1,Maybelline,Lips,Maybelline Lip Studio Color Jolt Shiny + Matte...,9.9,naivialyntalde,i chooose the colour very dull i dont like to ...,Stripped Dow,4.0,2019-11-21 13:05:00,no
2,2,Maybelline,Lips,Maybelline Lip Studio Color Jolt Shiny + Matte...,9.9,izzahxx,shipping took one week. pretty but will have t...,Violet Rebel,5.0,2019-11-21 19:12:00,no
3,3,Maybelline,Lips,Maybelline Lip Studio Color Jolt Shiny + Matte...,9.9,celestsoo,packing leak,Stripped Dow,1.0,2019-11-25 14:00:00,no
4,4,Maybelline,Lips,Maybelline Lip Studio Color Jolt Shiny + Matte...,9.9,sneaihasaravanan,really good quality and shade worth it’s price,Violet Rebel,5.0,2019-11-21 15:57:00,no


### Transformation and Initalise of data

In [4]:
# make the old csv data into a list
brand_list = data['Brand'].tolist()
category_list = data['Category'].tolist()
product_name_list = data['Product_Name'].tolist()
price_list = data['Price'].tolist()
reviewer_list = data['Reviewer'].tolist()
review_list = data['Review'].tolist()
product_variation_list = data['Product_Purchase'].tolist()
rating_list = data['Ratings'].tolist()
date_review_list = data["Date_Of_Review"].tolist()
response_list = data["Response"].tolist()

# initialising the new columns
brand_csv = []
category_csv = []
product_name_csv = []
prices_csv = []
reviewer_csv =[]
review_csv = []
product_variation_csv = []
rating_csv = []
date_review_csv = []
response_csv = []

# this is the length of the csv old data
print ("The length of the dataset is:", len(brand_list))

The length of the dataset is: 2422


### Spilt the Sentence 

In [5]:
for i in range(len(brand_list)):
    # current_review is a string
    current_review = review_list[i]
    #print(current_review)
    
    splitted_sentence= re.split(' and | but |\. | however |\,|\! ', current_review)
    
    for j in range(len(splitted_sentence)):
        if len(splitted_sentence[j]) > 0 :
            
            processed_review = splitted_sentence[j]
            # Remove all the special characters
            processed_review = re.sub(r'\W', ' ', processed_review)
            # remove all single characters
            processed_review= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_review)
            # Remove single characters from the start
            processed_review = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_review) 
            # Substituting multiple spaces with single space
            processed_review = re.sub(r'\s+', ' ', processed_review, flags=re.I)
            # Removing prefixed 'b'
            processed_review = re.sub(r'^b\s+', '', processed_review)
            
            # append into a dictionary
            brand_csv.append(brand_list[i])
            category_csv.append(category_list[i])
            product_name_csv.append(product_name_list[i])
            prices_csv.append(price_list[i])
            reviewer_csv.append(reviewer_list[i])
            review_csv.append(processed_review)
            product_variation_csv.append(product_variation_list[i])
            rating_csv.append(rating_list[i])
            date_review_csv.append(date_review_list[i])
            response_csv.append(response_list[i])
            
#Store into a new data frame
splitted_data = {'Brand':brand_csv, 'Category': category_csv, 'Product Name ': product_name_csv, 'Price':prices_csv ,'Reviewer':reviewer_csv,'Review':review_csv, 'Product Purchase':product_variation_csv,'Ratings':rating_csv,'Date Of Review':date_review_csv,'Response': response_csv }
splitted_df = pd.DataFrame.from_dict(splitted_data)
splitted_df.head()

,Brand,Category,Product Name,Price,Reviewer,Review,Product Purchase,Ratings,Date Of Review,Response
0,Maybelline,Lips,Maybelline Lip Studio Color Jolt Shiny + Matte...,9.9,peachiii,fast delivery,Bittersweet Mand,5.0,2019-11-24 14:32:00,no
1,Maybelline,Lips,Maybelline Lip Studio Color Jolt Shiny + Matte...,9.9,peachiii,bubble wrapped,Bittersweet Mand,5.0,2019-11-24 14:32:00,no
2,Maybelline,Lips,Maybelline Lip Studio Color Jolt Shiny + Matte...,9.9,peachiii,i have yet to try it out,Bittersweet Mand,5.0,2019-11-24 14:32:00,no
3,Maybelline,Lips,Maybelline Lip Studio Color Jolt Shiny + Matte...,9.9,naivialyntalde,i chooose the colour very dull dont like to us...,Stripped Dow,4.0,2019-11-21 13:05:00,no
4,Maybelline,Lips,Maybelline Lip Studio Color Jolt Shiny + Matte...,9.9,naivialyntalde,but the get on flash deal so still worth it th...,Stripped Dow,4.0,2019-11-21 13:05:00,no
